## Basic setting up

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy

In [2]:
listings = pd.read_csv("barcelona_listings.csv")

In [3]:
engine = create_engine("sqlite:///project.db")

## Import accommodates

In [4]:
df = pd.DataFrame(listings.accommodates.unique(), columns=["accommodates"])

In [5]:
df.to_sql("Accommodation", engine, if_exists="append", index=False, dtype={"accommodates" : sqlalchemy.CHAR(32)})

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: Accommodation.accommodates [SQL: 'INSERT INTO "Accommodation" (accommodates) VALUES (?)'] [parameters: ((6,), (8,), (2,), (3,), (4,), (5,), (10,), (1,)  ... displaying 10 of 16 total bound parameter sets ...  (14,), (15,))] (Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
engine.execute("select count(*) from Accommodation ").fetchall()

## Import amenities

In [83]:
amenities = pd.DataFrame(listings.amenities.unique(), columns=["amenities"])

In [84]:
temp = []
for index, row in amenities.iterrows():
    temp.append(row["amenities"].split(','))

In [85]:
flat_list = []
for sublist in temp:
    for item in sublist:
        flat_list.append(item)

In [86]:
def remove_chars(item):
    for c in item:
        if c == "{" or c == "}" or c == "'" or c == '"':
            item = item.replace(c, "")
        item = item.strip().lower()
    return item

In [87]:
new_list = []
for item in flat_list:
    new_list.append('"' + remove_chars(item) + '"')
    

In [88]:
new_list = list(set(new_list))

In [90]:
new_list.remove('""')

In [91]:
new_list

['"espresso machine"',
 '"handheld shower head"',
 '"carbon monoxide detector"',
 '"stair gates"',
 '"elevator"',
 '"pets live on this property"',
 '"baby monitor"',
 '"ski-in/ski-out"',
 '"disabled parking spot"',
 '"building staff"',
 '"coffee maker"',
 '"private pool"',
 '"smart lock"',
 '"gym"',
 '"day bed"',
 '"accessible-height bed"',
 '"warming drawer"',
 '"shared pool"',
 '"microwave"',
 '"cooking basics"',
 '"wheelchair accessible"',
 '"pack ’n play/travel crib"',
 '"electric profiling bed"',
 '"free street parking"',
 '"balcony"',
 '"pillow-top mattress"',
 '"bidet"',
 '"air conditioning"',
 '"hbo go"',
 '"baby bath"',
 '"washer"',
 '"private living room"',
 '"single level home"',
 '"beachfront"',
 '"outdoor parking"',
 '"netflix"',
 '"dishes and silverware"',
 '"bath towel"',
 '"cat(s)"',
 '"sun loungers"',
 '"table corner guards"',
 '"changing table"',
 '"bbq grill"',
 '"dishwasher"',
 '"luggage dropoff allowed"',
 '"memory foam mattress"',
 '"free parking on premises"',
 '

In [92]:
amenities = pd.DataFrame(new_list, columns=["amenities"])

In [93]:
amenities.to_sql("Amenities", engine, if_exists="append", index=False, dtype={"amenities" : sqlalchemy.CHAR(32)})

In [24]:
listings.id.count()

13095

In [29]:
len(listings["amenities"].unique() == listings.amenities.unique())

11837

## Import Bathrooms

In [61]:
bathrooms = pd.DataFrame(listings.bathrooms.unique(), columns=["bathrooms"])

In [62]:
bathrooms = bathrooms.dropna()

In [63]:
bathrooms.to_sql("Bathrooms", engine, if_exists="append", index=False, dtype={"bathrooms" : sqlalchemy.FLOAT})